In [ ]:
# 날짜 리스트에서 반복문 돌려서 가져온 다음 데이터프레임으로 합치고, csv로 저장하고, db에도 입력을 하게끔.

In [14]:
# 필요한 라이브러리 가져오기
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd

# 서비스 계정 인증
credentials = service_account.Credentials.from_service_account_file(
    'C:/phytoway_2/코드/python/데이터수집/datacollectautomation-04543e89aa13.json', scopes=['https://www.googleapis.com/auth/analytics.readonly'])

# Google Analytics Reporting API V4 빌드
analytics = build('analyticsreporting', 'v4', credentials=credentials)

# Google Analytics에서 데이터 조회를 위한 보고서 요청 생성
response = analytics.reports().batchGet(
    body={
        'reportRequests': [
            {
                'viewId': '253559313', # 조회 대상 뷰 ID 입력
                'dateRanges': [{'startDate': '2023-01-01', 'endDate': '2023-01-31'}], # 조회 기간 설정
                'metrics': [{'expression': 'ga:users'}, {'expression': 'ga:newUsers'}, {'expression': 'ga:sessions'}, {'expression': 'ga:bounceRate'}, {'expression': 'ga:pageviewsPerSession'}, {'expression': 'ga:avgSessionDuration'}, {'expression': 'ga:transactionsPerSession'}, {'expression': 'ga:transactions'}, {'expression': 'ga:transactionRevenue'}], # 조회할 지표 설정
                'dimensions': [{'name': 'ga:adGroup'}, {'name': 'ga:adwordsAdGroupID'}], # 조회할 차원 설정
                'pageSize': 5000, # 페이지당 결과 수 설정
                'samplingLevel': 'LARGE' # 샘플링 레벨 설정
            }]
    }
).execute()



# Google Analytics 응답에서 데이터 추출
rows = response['reports'][0]['data']['rows']
data = []
for row in rows:
    dimensions = row['dimensions']
    metrics = row['metrics'][0]['values']
    data.append(dimensions + metrics)

# 추출된 데이터를 Pandas DataFrame으로 변환
df = pd.DataFrame(data, columns=['ga:adGroup', 'ga:adwordsAdGroupID', 'Users', 'newUsers', 'Sessions', 'bounceRate', 'pageviewsPerSession', 'avgSessionDuration', 'transactionsPerSession', 'transactions', 'transactionRevenue'])


In [34]:
df[(df['transactions'] > '0') & (df['ga:adwordsAdGroupID'] != '(not set)')]

,ga:adGroup,ga:adwordsAdGroupID,Users,newUsers,Sessions,bounceRate,pageviewsPerSession,avgSessionDuration,transactionsPerSession,transactions,transactionRevenue
2,★쇼핑 캠페인,143535828146,3,1,3,33.33333333333333,8.0,175.0,66.66666666666666,2,114000.0
3,221017_써큐시안_남녀35이상,139398233941,1223,918,1334,39.20539730134933,2.082458770614693,27.04647676161919,0.8245877061469266,11,874500.0
4,221017_써큐시안_남녀45이상,139398233941,6987,5487,7780,39.39588688946015,2.0109254498714653,24.260796915167095,0.5269922879177378,41,2941500.0
6,221025_써큐시안_45세이상_조여사,142603603853,8588,6674,10302,38.71092991652107,1.9907784896136673,24.69559308872064,0.5824111822947,60,4411500.0
9,221121_판토모나_남녀35이상,141487919417,756,571,810,33.827160493827165,2.1604938271604937,38.635802469135804,0.7407407407407408,6,536000.0
10,221121_판토모나_남녀35이상_인터뷰,141487919417,4183,3141,4776,36.118090452261306,2.0814489112227808,28.07893634840871,0.6909547738693468,33,1923500.0
12,221129_써큐시안_리타겟팅,143641922123,2278,1847,2871,18.599791013584117,2.2361546499477534,27.794496691048415,0.9056078021595262,26,2300500.0
15,221129_판토모나_리타겟팅,143641922283,2436,1756,3228,15.272614622057002,2.4928748451053284,36.27695167286245,1.734820322180917,56,3628800.0
16,221130_파이토웨이_쇼핑,143535828146,1474,1383,1827,15.38040503557745,2.6639299397920086,49.65462506841817,2.736726874657909,50,2141700.0
17,221209_페미론큐_25이상(실험),145190245198,1225,894,1595,18.808777429467085,2.3586206896551722,43.08150470219436,2.25705329153605,36,2709000.0
